# Koiter Shells via Hellan-Herrmann-Johnson (HHJ) method
We start with the following three-field formulation [<a href="https://doi.org/10.1016/j.compstruc.2024.107543">Neunteufel, Schöberl. The Hellan-Herrmann-Johnson and TDNNS method for linear and nonlinear shells, Computers & Structures, (2024).</a>] to incorporate the distributional extrinsic curvature difference of the initial and deformed shell configuration: 

Find $(u,\boldsymbol{\kappa}^{\mathrm{diff}},\boldsymbol{\sigma})\in V_h^k\times M_h^{k-1,\mathrm{dc}}\times M_h^{k-1}$ for the Lagrangian 
\begin{align*}
		\mathcal{L}(u,\boldsymbol{\kappa}^{\mathrm{diff}},\boldsymbol{\sigma})&=\frac{t}{2}\|\boldsymbol{E}(u)\|_{\mathbb{M}}^2+\frac{t^3}{12}\|\boldsymbol{\kappa}^{\mathrm{diff}}\|_{\mathbb{M}}^2-\langle f,u\rangle+\sum_{T\in\mathcal{T}_h}\int_T\big(\boldsymbol{\kappa}^{\mathrm{diff}}-(\boldsymbol{F}^T\nabla_S(\nu\circ\phi)-\nabla_S\hat{\nu}) \big):\boldsymbol{\sigma}\,ds
		\\
		&\qquad+\sum_{E\in\mathcal{E}_h}\int_{E}(\sphericalangle(\nu_L\circ\phi,\nu_R\circ\phi)-\sphericalangle(\hat{\nu}_L,\hat{\nu}_R))\boldsymbol{\sigma}_{\hat{\mu}\hat{\mu}}\,dl.
	\end{align*}
    Here, $\boldsymbol{E}=\frac{1}{2}(\boldsymbol{F}^\top\boldsymbol{F}-\boldsymbol{P})= \frac{1}{2}(\nabla_S u^\top \nabla_S u + \nabla_S u^\top\boldsymbol{P} + \boldsymbol{P}\nabla_S u)$ denotes the Green-strain tensor restricting on the tangent space measuring the membrane energy of the shell, $t$ the thickness, and $\mathbb{M}$ the material tensor. $\nu$ and $\hat{\nu}$ are the normal vectors with respect to the deformed and initial configuration, respectively. $\hat{\mu}$ is the co-normal (element-normal) vector.
    
<center>
<img src="nv_conv_tang_trig.png" width="200"> 
</center>

With this formulation we circumvented the fourth-order problem by means of a mixed one and are able to compute the bending energy also on affine triangulations thanks to the edge terms measuring the angle difference between the initial and deformed configuration.

For an invertible material law, we can eliminate $\boldsymbol{\kappa}^{\mathrm{diff}}$ leading to a mixed saddle point problem in the displacement $u$ and moment tensor $\boldsymbol{\sigma}$. The term $\boldsymbol{F}^T\nabla_S(\nu\circ\phi)-\nabla_S\hat{\nu}$ can be rewritten and simplified. The following formulation can be seen as an extension of the Hellan-Herrmann-Johnson method from linear plates to nonlinear shells [<a href="https://doi.org/10.1016/j.compstruc.2019.106109">Neunteufel, Schöberl: The Hellan-Herrmann-Johnson method for nonlinear shell, <i>Comput. Struct.</i> (2019).</a>].

Find $(u,\boldsymbol{\sigma})\in V_h^k\times M_h^{k-1}$ for the Lagrangian 
\begin{align*}
\mathcal{L}(u,\sigma) &=\frac{t}{2}\|\boldsymbol{E}(u)\|^2_{\mathbb{M}} -\frac{6}{t^3}\|\boldsymbol{\sigma}\|^2_{\mathbb{M}^{-1}} +  \sum_{T\in\mathcal{T}_h}\int_{T} \boldsymbol{\sigma}:(\boldsymbol{H}_{\nu\circ\phi}+(1-\hat{\nu}\cdot\nu\circ\phi)\nabla_S\hat{\nu})\,ds \\
&\qquad+ \sum_{E\in\mathcal{E}_h}\int_E(\sphericalangle(\nu_L\circ\phi,\nu_R\circ\phi)-\sphericalangle(\hat{\nu_L},\hat{\nu_R}))\boldsymbol{\sigma}_{\hat{\mu}\hat{\mu}}\,dl - \int_{\mathcal{S}}f\cdot u\,ds,
\end{align*}
where $\boldsymbol{H}_{\nu\circ\phi}=\sum_{i=1}^3(\nabla_S^2u_i)\nu_i\circ\phi$, and $\nabla^2_S u_i=\boldsymbol{P}\nabla_S(\nabla_S u_i)$ denotes the Riemann surface Hessian.


### Cylindrical shell under volume load
As a first example we consider a cylindrical shell, which is clamped at the left side and free at the right side, and applying a volume force.

In [ ]:
from netgen.occ import *
from ngsolve import *
from ngsolve.webgui import Draw

order = 3

cyl = Cylinder((0, 0, 0), (1, 0, 0), 0.4, 1).faces[0]
cyl.edges.Min(X).name = "left"
cyl.edges.Max(X).name = "right"
mesh = Mesh(OCCGeometry(cyl).GenerateMesh(maxh=0.15)).Curve(order)
Draw(mesh)

# Young modulus and Poisson's ratio
E, nu = 2e1, 0.1
thickness = 0.02

At the clamped boundary we need to fix the displacement by homogeneous Dirichlet boundary conditions. As usual for mixed formulations the essential and natural boundary conditions swap for the stress field. Therefore, we have to set the essential boundary condition $\sigma_{\hat\mu\hat\mu}$ at the free boundary. $\sigma_{\hat\mu\hat\mu}=0$ has the physical meaning that no moment-force is applied at this boundary.

In [ ]:
fes1 = HDivDivSurface(mesh, order=order - 1, dirichlet_bbnd="right")
fes2 = VectorH1(mesh, order=order, dirichlet_bbnd="left")
fes = fes2 * fes1
u, sigma = fes.TrialFunction()
# need the trace as we are on a surface
sigma = sigma.Trace()

solution = GridFunction(fes, name="solution")

We define the outer normal vector $\hat\nu$, tangential vector $\hat t$ and the co-normal vector $\hat\mu = \hat\nu\times \hat t$ at the initial configuration.

<center>
<img src="nv_conv_tang_trig.png" width="200"> 
</center>

Then the projection operator onto the tangent space, deformation gradient, Cauchy-Green, and Green tensors $\boldsymbol{P}$, $\boldsymbol{F}$, $\boldsymbol{C}$, and $\boldsymbol{E}$ are defined.

Finally, the deformed normal and tangential vectors are declared, which depend on the unknown displacement field $u$.

In [ ]:
nsurf = specialcf.normal(mesh.dim)
t = specialcf.tangential(mesh.dim)
nel = Cross(nsurf, t)

Ptau = Id(mesh.dim) - OuterProduct(nsurf, nsurf)
Ftau = Grad(u).Trace() + Ptau
Ctau = Ftau.trans * Ftau
Etautau = 0.5 * (Ctau - Ptau)

# Normal vector transforms with cofactor matrix, edge tangent vector with push forward F
nphys = Normalize(Cof(Ftau) * nsurf)
tphys = Normalize(Ftau * t)
nelphys = Cross(nphys, tphys)

gradn = specialcf.Weingarten(3)  # Grad(nsurf)
Hn = CF((u.Operator("hesseboundary").trans * nphys), dims=(3, 3))

For the angle computation of the bending energy we use an averaged normal vector avoiding the necessity of using information of two neighbored element at once (+ a more stable formulation using the co-normal vector instead of the outer normal vector)

<center>
<img src="nonsmooth_av_nv_el_nv.png" width="150"> 
</center>

\begin{align*}
\sum_{E\in\mathcal{E}_h}\int_E(\sphericalangle(\nu_L\circ\phi,\nu_R\circ\phi)-\sphericalangle(\hat{\nu}_L,\hat{\nu}_R))\boldsymbol{\sigma}_{\hat{\mu}\hat{\mu}}\,dl &= \sum_{T\in\mathcal{T}_h}\int_{\partial T}(\sphericalangle(\nu\circ\phi,\{\!\{\nu\circ\phi\}\!\})-\sphericalangle(\hat{\nu},\{\!\{\hat{\nu}\}\!\}))\boldsymbol{\sigma}_{\hat{\mu}\hat{\mu}}\,dl\\
&= \sum_{T\in\mathcal{T}_h}\int_{\partial T}(\sphericalangle(\hat{\mu},\{\!\{\hat{\nu}\}\!\})-\sphericalangle(\mu\circ\phi,\{\!\{\nu\circ\phi\}\!\}))\boldsymbol{\sigma}_{\hat{\mu}\hat{\mu}}\,dl\\
&= \sum_{T\in\mathcal{T}_h}\int_{\partial T}(\sphericalangle(\hat{\mu},\{\!\{\hat{\nu}\}\!\})-\sphericalangle(\mu\circ\phi,P^{\perp}_{\tau\circ\phi}(\{\!\{\nu^n\}\!\})))\boldsymbol{\sigma}_{\hat{\mu}\hat{\mu}}\,dl,
\end{align*}
where 
$$
P^{\perp}_{\tau\circ\phi}(v):= \frac{1}{\|\boldsymbol{P}^{\perp}_{\tau\circ\phi}v\|}\boldsymbol{P}^{\perp}_{\tau\circ\phi}v,\qquad \boldsymbol{P}^{\perp}_{\tau\circ\phi}:= \boldsymbol{I}-\tau\circ\phi\otimes\tau\circ\phi
$$
denotes the (nonlinear) normalized projection to the plane perpendicular to the deformed edge tangential vector for measuring the correct angle.

In [ ]:
gfclamped = GridFunction(FacetSurface(mesh, order=0))
gfclamped.Set(1, definedon=mesh.BBoundaries("left"))

fesVF = VectorFacetSurface(mesh, order=order)
averednv = GridFunction(fesVF)
averednv_start = GridFunction(fesVF)

cfnphys = Normalize(Cof(Ptau + Grad(solution.components[0])) * nsurf)

cfnR = Normalize(averednv_start)
pnaverage = Normalize(averednv - (tphys * averednv) * tphys)

averednv.Set(nsurf, dual=True, definedon=mesh.Boundaries(".*"))
averednv_start.Set(nsurf, dual=True, definedon=mesh.Boundaries(".*"))

Define the material norms $\|\cdot\|_{\mathbb{M}}^2$ and $\|\cdot\|_{\mathbb{M}^{-1}}^2$ with Young modulus $\bar{E}$ and Poisson's ration $\bar{\nu}$
\begin{align*}
\mathbb{M} \boldsymbol{E} = \frac{\bar E}{1-\bar \nu^2}\big((1-\bar \nu)\boldsymbol{E}+\bar \nu\,\mathrm{tr}(\boldsymbol{E})\boldsymbol{P}\big)
\end{align*}

In [ ]:
def MaterialNorm(mat, E, nu):
    return E / (1 - nu**2) * ((1 - nu) * InnerProduct(mat, mat) + nu * Trace(mat) ** 2)


def MaterialNormInv(mat, E, nu):
    return (1 + nu) / E * (InnerProduct(mat, mat) - nu / (nu + 1) * Trace(mat) ** 2)

Define the bilinear form for the problem including membrane and bending energy
\begin{align*}
\mathcal{L}(u,\sigma) &=\frac{t}{2}\|\boldsymbol{E}(u)\|^2_{\mathbb{M}} -\frac{6}{t^3}\|\boldsymbol{\sigma}\|^2_{\mathbb{M}^{-1}} +  \sum_{T\in\mathcal{T}_h}\Big(\int_{T} \boldsymbol{\sigma}:(\boldsymbol{H}_{\nu\circ\phi}+(1-\hat{\nu}\cdot\nu\circ\phi)\nabla_S\hat{\nu})\,ds \\
&\qquad+ \int_{\partial T}(\sphericalangle(\mu\circ\phi,\{\!\{\nu^n\}\!\})-\sphericalangle(\hat{\mu},\{\!\{\hat{\nu}\}\!\}))\boldsymbol{\sigma}_{\hat{\mu}\hat{\mu}}\,dl\Big) - \int_{\mathcal{S}}f\cdot u\,ds.
\end{align*}

In [ ]:
bfA = BilinearForm(fes, symmetric=True, condense=True)
# membrane energy
bfA += Variation(0.5 * thickness * MaterialNorm(Etautau, E, nu) * ds).Compile()
# bending energy via HHJ
bfA += Variation(
    (
        -6 / thickness**3 * MaterialNormInv(sigma, E, nu)
        + InnerProduct(Hn + (1 - nsurf * nphys) * gradn, sigma)
    )
    * ds
).Compile()
# boundary term of bending energy (change of normal vectors at element interfaces)
bfA += Variation(
    (acos(nelphys * pnaverage) - acos(nel * cfnR))
    * sigma[nel,nel]
    * ds(element_boundary=True)
).Compile()

# body force as right-hand side
par = Parameter(0.0)
bfA += Variation(-2*thickness * par * y * u[1] * ds)

In [ ]:
solution.vec[:] = 0
scene = Draw(solution.components[0], mesh, "disp", deformation=solution.components[0])

Use Newton's method for solving and increase magnitude of right-hand side by load-steps.

The normal vector needs to be averaged after each Newton step.

In [ ]:
numsteps = 10
with TaskManager():
    for steps in range(numsteps):
        par.Set((steps + 1) / numsteps)
        print("Loadstep =", steps + 1, ", F/Fmax =", (steps + 1) / numsteps * 100, "%")

        # update averaged normal vector
        averednv.Set(
            (1 - gfclamped) * cfnphys + gfclamped * nsurf,
            dual=True,
            definedon=mesh.Boundaries(".*"),
        )

        # solve
        solvers.Newton(
            bfA, solution, inverse="umfpack", printing=False, maxerr=1e-10, maxit=20
        )
        scene.Redraw()

Advantage:
* No $H^2$ finite elements needed

Disadvantage:
* Saddle point problem
* Moments are prescribed as essential Dirichlet data, not optimal for load-steps with moments as right-hand side

Therefore we use hybridization making $\sigma$ discontinuous and reinforcing the normal-normal continuity by a Lagrange multiplier $\alpha$.

<center>
<table><tr>
<td> <img src="h1_p1_trig.png" width="150"/> </td>
<td> <img src="hdivdiv_0_trig_2.png" width="170"/> </td>
<td> <img src="normalfacet_0_trig_0.png" width="150"/> </td>
</tr></table>
</center>

This enables also to statically condense out $\sigma$ and the resulting system in $(u,\alpha)$ is again a minimization problem such that we can use e.g. sparsecholesky solver (or CG). The resulting dofs are equivalent to the famous Morley triangle which is a non-conforming element for fourth order problems.
<center>
<img src="morley_element.png" width="150"/>
</center>

### Cantilever with bending moments
<center>
<img src="cant_bend_mom_1d.png" width="200"/>
</center>

We consider a beam which is fixed at the left boundary and we will apply a moment at the right boundary such that the beam should roll up to a circle (Possion ratio $\bar\nu=0$). We use loadsteps to increase the moments and apply Newton's method. As the bending moment would be incorporated strongly via $\sigma_{\hat\mu\hat\mu}$, which is tedious, we use the hybridized formulation such that we can include the force weakly directly in the formulation. 

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *

thickness = 0.1
L = 12
W = 1
E, nu = 1.2e6, 0
moment = IfPos(x - L + 1e-6, 1, 0) * 50 * pi / 3

def mapping(x, y, z): return (L * x, W * y, 0)
rect = Rectangle(L, W).Face()
rect.edges.Min(X).name = "left"
rect.edges.Max(X).name = "right"
rect.edges.Min(Y).name = "bottom"
rect.edges.Max(Y).name = "top"
mesh = Mesh(OCCGeometry(rect).GenerateMesh(maxh=1))
Draw(mesh);

Due to hybridization the essential and natural boundary conditions swapped again. For the Lagrange multiplier, which has the physical meaning of the rotated angle, we set homogeneous Dirichlet at the clamped left boundary.

In [ ]:
# try out order = 1, 2, 3 with and without Regge interpolation in the membrane energy term
order = 2

fes1 = HDivDivSurface(mesh, order=order - 1, discontinuous=True)
fes2 = VectorH1(
    mesh,
    order=order,
    dirichletx_bbnd="left",
    dirichlety_bbnd="left|bottom",
    dirichletz_bbnd="left",
)
fes3 = NormalFacetSurface(mesh, order=order - 1, dirichlet_bbnd="left")
fes = fes2 * fes1 * fes3
u, sigma, hyb = fes.TrialFunction()
# trace needed as we are on the surface
sigma, hyb = sigma.Trace(), hyb.Trace()

# For avoiding membrane locking for order >= 2
Regge = HCurlCurl(mesh, order=order - 1)

solution = GridFunction(fes, name="solution")

Define again the tensors and deformed vectors

In [ ]:
nsurf = specialcf.normal(mesh.dim)
t = specialcf.tangential(mesh.dim)
nel = Cross(nsurf, t)

Ptau = Id(mesh.dim) - OuterProduct(nsurf, nsurf)
Ftau = grad(u).Trace() + Ptau
Ctau = Ftau.trans * Ftau
Etautau = 0.5 * (Ctau - Ptau)

nphys = Normalize(Cof(Ftau) * nsurf)
tphys = Normalize(Ftau * t)
nelphys = Cross(nphys, tphys)

Hn = CF((u.Operator("hesseboundary").trans * nphys), dims=(3, 3))

Stuff for averaging normal vector and material laws

In [ ]:
gfclamped = GridFunction(FacetSurface(mesh, order=0))
gfclamped.Set(1, definedon=mesh.BBoundaries("left"))

fesVF = VectorFacetSurface(mesh, order=order)
averednv = GridFunction(fesVF)
averednv_start = GridFunction(fesVF)

cfnphys = Normalize(Cof(Ptau + grad(solution.components[0])) * nsurf)

cfnR = Normalize(averednv_start)
pnaverage = Normalize(averednv - (tphys * averednv) * tphys)

averednv.Set(nsurf, dual=True, definedon=mesh.Boundaries(".*"))
averednv_start.Set(nsurf, dual=True, definedon=mesh.Boundaries(".*"))


def MaterialNorm(mat, E, nu):
    return E / (1 - nu**2) * ((1 - nu) * InnerProduct(mat, mat) + nu * Trace(mat) ** 2)


def MaterialNormInv(mat, E, nu):
    return (1 + nu) / E * (InnerProduct(mat, mat) - nu / (nu + 1) * Trace(mat) ** 2)

Define hybridized energy and set condense=True to condense out bending moment unknown.

For $k\geq 2$ so-called membrane locking can occur for small thickness parameters due to the different scaling of the membrane and bending energy. To circumvent this locking problem we can interpolate the membrane strains into Regge finite elements of one order less than the displacement fields [<a href="https://doi.org/10.1016/j.cma.2020.113524">Neunteufel, Schöberl. Avoiding membrane locking with Regge interpolation. <i>Computer Methods in Applied Mechanics and Engineering</i>, (2021).</a>].

In [ ]:
bfA = BilinearForm(fes, symmetric=True, condense=True)
# membrane energy
if True:
    bfA += Variation(0.5 * thickness * MaterialNorm(Etautau, E, nu) * ds)
else:  # interpolate the membrane strains into Regge elements to avoid membrane locking
    bfA += Variation(
        0.5 * thickness * MaterialNorm(Interpolate(Etautau, Regge), E, nu) * ds
    )
# bending energy
bfA += Variation(
    (
        -6 / thickness**3 * MaterialNormInv(sigma, E, nu)
        + InnerProduct(Hn + (1 - nsurf * nphys) * Grad(nsurf), sigma)
    )
    * ds
).Compile()
# boundary term of bending energy including hybridization variable
bfA += Variation(
    (acos(nelphys * pnaverage) - acos(nel * cfnR) + hyb * nel)
    * sigma[nel,nel]
    * ds(element_boundary=True)
).Compile()

# moment as right-hand side
par = Parameter(0.0)
bfA += Variation(-par * moment * (hyb * nel) * ds(element_boundary=True))

In [ ]:
solution.vec[:] = 0
scene = Draw(solution.components[0], mesh, "disp", deformation=solution.components[0])

Average new normal vector and solve with Newton.

In [ ]:
numsteps = 10
with TaskManager():
    for steps in range(numsteps):
        par.Set((steps + 1) / numsteps)
        print("Loadstep =", steps + 1, ", F/Fmax =", (steps + 1) / numsteps * 100, "%")

        # update averaged normal vector
        averednv.Set(
            (1 - gfclamped) * cfnphys + gfclamped * nsurf,
            dual=True,
            definedon=mesh.Boundaries(".*"),
        )

        solvers.Newton(
            bfA, solution, inverse="sparsecholesky", printing=False, maxerr=1e-10
        )
        scene.Redraw()

It works perfectly for lowest order (linear displacement). Increasing to second order the cantilever does not completely roll up to a circle, why?

The observed behavior is due to membrane locking of shells. The usage of higher polynomial degree mitigates the problem.

We can try out how many rounds the shell can do by further increasing the bending moment.

In [ ]:
numsteps = 10
with TaskManager():
    for steps in range(numsteps):
        par.Set(par.Get() + (1) / numsteps)
        print("Loadstep =", steps + 1, ", F/Fmax =", (steps + 1) / numsteps * 100, "%")

        # update averaged normal vector
        averednv.Set(
            (1 - gfclamped) * cfnphys + gfclamped * nsurf,
            dual=True,
            definedon=mesh.Boundaries(".*"),
        )

        solvers.Newton(
            bfA, solution, inverse="sparsecholesky", printing=False, maxerr=1e-10
        )
        scene.Redraw()